In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import math
import scipy.optimize
import time

In [2]:
class Lattice:
    def printLattice(self):
        for t, level in enumerate(self.lattice):
            print 'level {0}'.format(t)
            level = [ round(elem, 3) for elem in level ]
            print ', '.join(map(str, level))

In [3]:
class CalibratedRateLattice(Lattice):
    def __init__(self, dP, vP, q):
        self.lattice = []
        self.lattice.append([dP[0]])
        for i in range(1,len(dP)):
            level = []
            for j in range(i+1):
                rate = dP[i]*math.exp(vP*j)
                level.append(rate)
            self.lattice.append(level)

In [4]:
class RateLattice(Lattice):    
    def __init__(self, n, S0, u, d):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                rate = S0 * u**j * d**(i - j)
                level.append(rate)
            self.lattice.append(level)

In [5]:
class HazardLattice(Lattice):    
    def __init__(self, n, a, b):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                rate = a*b**(j-.5*i)
                level.append(rate)
            self.lattice.append(level)

In [6]:
class ZCBLattice(Lattice):
    def __init__(self, F, q, n, rateLattice, hazardLattice, recoveryRate):
        self.lattice = []
        clippedRate = rateLattice[:n+1]
        clippedHazard = hazardLattice[:n+1]
        rightLevel = []
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
#                     hazard = clippedHazard[n-i][j]
#                     nondefaultValue = (1.-hazard)*F
#                     defaultValue = hazard*recoveryRate*F
                    newLevel.append(F)
            else:
                for j in range(len(level)):
                    discount = 1.+clippedRate[n-i][j]/100.
                    hazard = clippedHazard[n-i][j]
                    nondefaultValue = (q*rightLevel[j+1]+(1.-q)*rightLevel[j])*(1.-hazard)
                    defaultValue = hazard*recoveryRate*F
                    price = (nondefaultValue+defaultValue)/discount
                    newLevel.append(price)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [7]:
class EPLattice(Lattice):
    def __init__(self, F, rateLattice):
        self.lattice = []
        for i in range(len(rateLattice)+1):
            newLevel = []
            if i == 0:
                newLevel.append(F)
            else:
                level = rateLattice[i-1]
                for j in range(len(level)+1):
                    if j == 0:
                        discount = 1.+level[j]/100.
                        price = .5*leftLevel[j]/discount
                    elif j == len(level):
                        discount = 1.+level[j-1]/100.
                        price = .5*leftLevel[j-1]/discount
                    else:
                        discount = 1.+level[j]/100.
                        price = .5*leftLevel[j]/discount
                        discount = 1.+level[j-1]/100.
                        price += .5*leftLevel[j-1]/discount
                    newLevel.append(price)
            leftLevel = newLevel
            self.lattice.append(newLevel)
    
    def getZCBPrices(self):
        return [sum(arr) for arr in self.lattice]
    
    def getSpotRates(self):
        zcbPrices= self.getZCBPrices()
        return [100.*((1./price)**(1./i)-1) for i, price in enumerate(zcbPrices) if i>0]

In [8]:
class SwapLattice(Lattice):
    def __init__(self, q, n, rf, firstPaymentTime, payFixed, rateLattice):
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print "Calculating swaps"
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    payment = spotRate-rf if payFixed else rf-spotRate
                    newPrice = payment/(1+spotRate)
                    newLevel.append(newPrice)
            else:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    newPrice = (q*rightLevel[j+1]+(1-q)*rightLevel[j])/(1.+spotRate)
                    if n-i >= firstPaymentTime:
                        payment = spotRate-rf if payFixed else rf-spotRate
                        newPrice += payment/(1.+spotRate)
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [9]:
class OptionLattice(Lattice):
    def __init__(self, n, q, K, isCall, isAmerican, rateLattice, baseLattice):
        multiplier = 1 if isCall else -1
        clippedBase = baseLattice[:n+1]
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print "Calculating options"
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    newLevel.append(max(multiplier * (level[j]-K), 0))
            else:
                for j in range(len(level)):
                    earlyExercise = max(multiplier * (level[j]-K), 0)
                    discount = 1.+clippedRate[n-i][j]/100.
                    hold = (q*rightLevel[j+1] + (1-q)*rightLevel[j])/discount
                    if earlyExercise > hold and isAmerican:
                        print "At time {0}, it's better to early exercise {1} than hold {2}".format(n-i, earlyExercise, hold)
                    newPrice = max(hold, earlyExercise) if isAmerican else hold
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [10]:
def G(x):
    rL = CalibratedRateLattice(x, volatilityParam, upMoveChance)
    epL = EPLattice(faceValue, rL.lattice[:])
    return [a - b for a, b in zip(epL.getSpotRates(), marketSpotRates)]

In [11]:
driftParams = [5.]*14
volatilityParam = .005
upMoveChance = .5

rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)

faceValue = 1.

epL = EPLattice(faceValue, rL.lattice[:])

modelZCBPrices = epL.getZCBPrices()

modelSpotRates = epL.getSpotRates()

marketSpotRates = [7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22,11.55,11.92,12.2,12.32]

print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

start_time = time.time()
maxIterations = 200
driftParams = scipy.optimize.broyden1(G, driftParams, iter=maxIterations)
elapsed_time = time.time() - start_time

print
print "%.2f ms elapsed for %d iterations" % (elapsed_time*1000, maxIterations)
print
print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

spot rate squared error
389.769846071

357.60 ms elapsed for 200 iterations

spot rate squared error
2.37436302008e-19


In [12]:
# Questions 1 and 2 should be answered by building and calibrating a 10-period Black-Derman-Toy model 
# for the short-rate, ri,j. You may assume that the term-structure of interest rates observed 
# in the market place is:

# Spot Rates 3.0% 3.1% 3.2% 3.3% 3.4% 3.5% 3.55% 3.6% 3.65% 3.7%

# As in the video modules, these interest rates assume per-period compounding so that, 
# for example, the market-price of a zero-coupon bond that matures in period 6
# is Z60=100/(1+.035)6=81.35 assuming a face value of 100.

# _____________________________________________________________________

# Questions 3-5 refer to the material on defaultable bonds and credit-default swaps (CDS).

In [13]:
# Q1
# Assume b=0.05 is a constant for all i in the BDT model 
# Calibrate the ai parameters so that the model term-structure matches the market term-structure

# Once your model has been calibrated, compute the price of a payer swaption 
# with notional $1M that expires at time t=3 with an option strike of 0. 
# You may assume the underlying swap has a fixed rate of 3.9% 
# and that if the option is exercised then cash-flows take place at times t=4,…,10.
# (The cash-flow at time t=i is based on the short-rate that prevailed in the previous period,
#  i.e. the payments of the underlying swap are made in arrears.)
# 
# rounded to the nearest integer

numPeriods = 10
marketSpotRates = [3.0,3.1,3.2,3.3,3.4,3.5,3.55,3.6,3.65,3.7]
driftParams = [5.]*len(marketSpotRates)
volatilityParam = .05
upMoveChance = .5
faceValue = 1.
maxIterations = 200

print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

start_time = time.time()
driftParams = scipy.optimize.broyden1(G, driftParams, iter=maxIterations)
elapsed_time = time.time() - start_time

print
print "%.2f ms elapsed over %d iterations" % (elapsed_time*1000, maxIterations)
print
print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

print
print 'rate lattice'
rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)
# rL.printLattice()
print

numPeriods = 10
fixedRate = .039
firstPaymentTime = 4
paysFixed = True

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, paysFixed, rL.lattice[:])
print 'swap lattice'
print
# sL.printLattice()
# print

numPeriods = 3
strikePrice = 0.
isCall = True
isAmerican = False

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], sL.lattice[:])
print 'option lattice on swap'
print
# oL.printLattice()
# print
print 'price w/ notional of $1M'
print oL.lattice[0][0]*1000000

spot rate squared error
36.14539126


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/site-packages/scipy/optimize/nonlin.py:997: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)
/usr/local/lib/python2.7/site-packages/scipy/optimize/nonlin.py:997: RuntimeWarning: divide by zero encountered in true_divide
  d = v / vdot(df, v)
/usr/local/lib/python2.7/site-packages/scipy/optimize/nonlin.py:773: RuntimeWarning: invalid value encountered in multiply
  self.collapsed += c[:,None] * d[None,:].conj()



260.64 ms elapsed over 200 iterations

spot rate squared error
7.34493163836e-24

rate lattice

Calculating swaps
swap lattice

Calculating options
option lattice on swap

price w/ notional of $1M
4102.11763759


In [14]:
# Q2
# Repeat the previous question but now assume a value of b=0.1
# 
# rounded to the nearest integer

numPeriods = 10
volatilityParam = .1
driftParams = [5.]*len(marketSpotRates)

print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

start_time = time.time()
driftParams = scipy.optimize.broyden1(G, driftParams, iter=maxIterations)
elapsed_time = time.time() - start_time

print
print "%.2f ms elapsed over %d iterations" % (elapsed_time*1000, maxIterations)
print
print 'spot rate squared error'
print sum(i*i for i in G(driftParams))

print
print 'rate lattice'
rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)
# rL.printLattice()
print

numPeriods = 10
fixedRate = .039
firstPaymentTime = 4
paysFixed = True

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, paysFixed, rL.lattice[:])
print 'swap lattice'
print
# sL.printLattice()
# print

numPeriods = 3
strikePrice = 0.
isCall = True
isAmerican = False

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], sL.lattice[:])
print 'option lattice on swap'
print
# oL.printLattice()
# print
print 'price w/ notional of $1M'
print oL.lattice[0][0]*1000000

spot rate squared error
50.7684224445

226.96 ms elapsed over 200 iterations

spot rate squared error
8.2491184859e-27

rate lattice

Calculating swaps
swap lattice

Calculating options
option lattice on swap

price w/ notional of $1M
8096.5697157


In [18]:
# Q3

# Construct a n=10-period binomial model for the short-rate, ri,j. 
# The lattice parameters are: r0,0=5%, u=1.1, d=0.9 and q=1−q=1/2.

# Assume that the 1-step hazard rate in node (i,j) is given by hij=a*b^(j−i/2) where a=0.01 and b=1.01.
# Compute the price of a zero-coupon bond with face value F=100 and recovery R=20%.

# rounded to two decimal places

numPeriods = 10
startRate = 5
upMoveReturn = 1.1
downMoveReturn = .9
upMoveChance = .5

rL = RateLattice(numPeriods, startRate, upMoveReturn, downMoveReturn)
print 'rate lattice'
print
# rL.printLattice()
# print

driftParam  = .01
volatilityParam = 1.01

hL = HazardLattice(numPeriods, driftParam, volatilityParam)
print 'hazard lattice'
print
# hL.printLattice()
# print

faceValue = 100
recoveryRate = .2
zL = ZCBLattice(faceValue, upMoveChance, numPeriods, rL.lattice[:], hL.lattice[:], recoveryRate)
print 'zcb lattice'
print
zL.printLattice()

rate lattice

hazard lattice

zcb lattice

level 0
57.217
level 1
63.034, 57.931
level 2
68.595, 64.067, 59.003
level 3
73.838, 69.936, 65.496, 60.517
level 4
78.72, 75.464, 71.705, 67.416, 62.587
level 5
83.217, 80.602, 77.547, 74.007, 69.95, 65.358
level 6
87.319, 85.322, 82.963, 80.196, 76.975, 73.262, 69.029
level 7
91.032, 89.612, 87.921, 85.916, 83.552, 80.783, 77.567, 73.869
level 8
94.368, 93.478, 92.409, 91.13, 89.606, 87.797, 85.662, 83.16, 80.251
level 9
97.349, 96.932, 96.429, 95.823, 95.093, 94.218, 93.172, 91.925, 90.447, 88.705
level 10
100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0


In [16]:
# Q4

# The true price of 5 different defaultable coupon paying bonds with non-zero recovery are specified in worksheet 
# 𝙲𝚊𝚕𝚒𝚋𝚛𝚊𝚝𝚒𝚘𝚗 in the workbook 𝙰𝚜𝚜𝚒𝚐𝚗𝚖𝚎𝚗𝚝𝟻_𝚌𝚍𝚜.𝚡𝚕𝚜𝚡. The interest rate is r=5% per annum. 
# Calibrate the six month hazard rates 𝙰𝟼 to 𝙰𝟷𝟼 to by minimizing the 𝚂𝚞𝚖𝙴𝚛𝚛𝚘𝚛 
# ensuring that the term structure of hazard rates are non-decreasing. You can model the non-decreasing
# hazard rates by adding constraints of the form 𝙰𝟼≤𝙰𝟽,…,𝙰𝟷𝟻≤𝙰𝟷𝟼. 
# Report the hazard rate at time 0 as a percentage.

# percent rounded to two decimal places

# 0.01844022368476 -> 1.84

In [17]:
# Q5

# Modify the data on the 𝙲𝙳𝚂𝚙𝚛𝚒𝚌𝚒𝚗𝚐 worksheet in the workbook 𝚋𝚘𝚗𝚍𝚜_𝚊𝚗𝚍_𝚌𝚍𝚜.𝚡𝚕𝚜𝚡 
# to compute a par spread in basis points for a 5yr CDS with notional principal N=10 million 
# assuming that the expected recovery rate R=25%, the 3-month hazard rate is a flat 1%,
# and the interest rate is 5% per annum.

# basis points rounded to two decimal places (1 bps = 0.01%)

# 301.51